# TRANSFER LEARNING

In [1]:
# downloading our data and becoming one with it.
import zipfile

# downloading the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# unzip the data
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2024-08-21 03:35:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  33.7MB/s    in 5.6s    

2024-08-21 03:35:23 (28.6 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
# how many images in each folder
import os
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/tes

# STEP 1:**Preparing Data for Modelling**

we will use ImageDataGenerator for loading or data into batches

In [3]:
#  end to end cnn model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# setting a random seed
tf.random.set_seed(42)


# processing data so they gets in range of 0 to 1...that's called scaling /normalizing

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# setup the train and test directory
train_dir="/content/10_food_classes_10_percent/train/"

test_dir="/content/10_food_classes_10_percent/test/"

# import data from directories and make it into batches
print("Training Images:")
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical",
                                             seed=42)

print("Testing Images")
test_data_10_percent=test_datagen.flow_from_directory(test_dir,
                                                  batch_size=32,
                                                  target_size=(224,224),
                                                  class_mode="categorical",
                                                  seed=42)



Training Images:
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


# CALLBACKS


In machine learning, a callback is a function or set of functions that are executed at specific stages during the training process of a model. Callbacks allow users to monitor and influence the behavior of a model as it trains, typically to perform tasks such as saving model checkpoints, adjusting learning rates, stopping training early if a metric (like loss) isn't improving, or logging metrics.
The following are the few callbacks:
1. Tensorboard Callback
2. ModelCheckpoint Callback
3. EarlyStopping CallBack

In [4]:
import datetime

In [5]:
# TensorBoard Callback
def create_tensorboard_callback(dir_name,eperiment_name):
  log_dir= dir_name + "/" + eperiment_name + datetime.datetime.now().strftime()
  TensorBoard_callabck=tensorflow.keras.Callback.TensorBoard(log_dir=log_dir)
  print(f"saving log_dir to:{log_dir}")
  return TensorBoard_callabck


# creating our model using TensorflowHub

In [6]:
 # Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"


What is a TensorFlow Hub?

Ans. TensorFlow Hub is a repository of reusable machine learning models. It allows developers to easily access and use pre-trained models, which can be integrated into their TensorFlow workflows. These models can be for tasks like image classification, object detection, text embeddings, and more.

In [7]:
# import few dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [8]:
def create_model(model_url, num_classes=10):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=(224,224,3)) # define the input image shape

  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer
  ])

  return model

In [12]:
# Create model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# Compile
resnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy,
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fe42970e710> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

# TRANSFER LEARNING : FINE TUNING
This time we are going to see how can we use the pretrained model with **tf.keras.applications** and applt them to our own problems.

**Getting our data**

In [13]:
# getting 10% of our data

In [16]:
import zipfile

In [18]:
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"

unzip_data("10_food_classes_10_percent.zip")


--2024-08-21 03:46:05--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.207, 172.217.218.207, 142.251.31.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.2’

10_food_classes_10_ 100%[===================>] 160.74M  41.6MB/s    in 4.1s    

2024-08-21 03:46:09 (38.9 MB/s) - ‘10_food_classes_10_percent.zip.2’ saved [168546183/168546183]



NameError: name 'unzip_data' is not defined